<a href="https://colab.research.google.com/github/smitmirani10/Sem3FinalProject/blob/main/Draft_project_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install torch
!pip install transformers
!pip install graphviz
!sudo apt-get install python-dev graphviz libgraphviz-dev pkg-config
!sudo pip install pygraphviz

Reading package lists... Done
Building dependency tree       
Reading state information... Done
pkg-config is already the newest version (0.29.1-0ubuntu2).
python-dev is already the newest version (2.7.15~rc1-1).
graphviz is already the newest version (2.40.1-2).
libgraphviz-dev is already the newest version (2.40.1-2).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [34]:
from transformers import AutoModel,AutoTokenizer
import pandas as pd
import numpy as np
import torch
import re
import networkx as nx
import pylab as plt
from networkx.drawing.nx_agraph import graphviz_layout, to_agraph
import pygraphviz as pgv
from sklearn.metrics import confusion_matrix,f1_score,accuracy_score
%pylab inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.7/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['text', 'plt', 'number']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [35]:
with open('/content/drive/MyDrive/CoNLL files/new_temp.conll', mode='r') as in_file, \
     open('/content/drive/MyDrive/CoNLL files/new.conll', mode='w') as out_file:
    x = in_file.readlines()
    for i in x:
      if i!='\n':
        out_file.write(i)
    in_file.close()
    out_file.close()

In [36]:
#reading a CoNLL file into a dataframe
#DF has 4 columns, 2 from the conll file and rest from the tokens and embeddings that we will generate
#1st 2 columns are the actual word and the coreference
#3rd & 4th words are the tokens generated from that word & their corresponding embeddings generated by BERT
in_file = open('/content/drive/MyDrive/CoNLL files/new.conll', 'r')
#out_file=open('/content/drive/MyDrive/CoNLL files/new.conll', mode='w')
count = 0
sentence_list =[]
coref_list=[]
token_list=[]
embedding_list=[]
column_names = ["words", "coref","tokens_per_word","embedding_per_token"]
df = pd.DataFrame(columns = column_names)
while True:
    count += 1
    # Get next line from file
    line = in_file.readline()
    if (count == 1):
      line = in_file.readline()
    #print(line)
    #zero out the tab count before starting to read every line
    tab_count=0
    # if line is empty
    # end of file is reached
    if not line.strip():
        break
    list_of_words = line.split()
    #print(list_of_words)
    sentence_list.append(list_of_words[3])
    coref_list.append(list_of_words[12])
    token_list.append(None)
    embedding_list.append(None)
in_file.close()
df['words'] = sentence_list
df['coref'] = coref_list
df['tokens_per_word'] = token_list
#df['tokens_per_word'] = ''
#df['tokens_per_word'] = df['tokens_per_word'].apply(list)

#df['embedding_per_token'] = ''
#df['embedding_per_token'] = df['embedding_per_token'].apply(list)
df['embedding_per_token'] = embedding_list
df

,words,coref,tokens_per_word,embedding_per_token
0,Diabetes,(1,None,None
1,Insipidus,-,None,None
2,in,-,None,None
3,Mice,(2,None,None
4,with,-,None,None
...,...,...,...,...
647,kidney,6),None,None
648,[,-,None,None
649,5,-,None,None
650,],-,None,None


In [37]:
PubMedBERT_fulltext = 'microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext'
BERT_base_uncased = 'bert-base-uncased'


In [38]:
#Transfer learning step
# Loading the pre-trained BERT model
###################################
# Embeddings will be derived from
# the outputs of this model
model = AutoModel.from_pretrained(PubMedBERT_fulltext,
                                  output_hidden_states = True,
                                  )

# Setting up the tokenizer
###################################
# This is the same tokenizer that
# was used in the model to generate 
# embeddings to ensure consistency
tokenizer = AutoTokenizer.from_pretrained(PubMedBERT_fulltext)

Some weights of the model checkpoint at microsoft/BiomedNLP-PubMedBERT-base-uncased-abstract-fulltext were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [39]:
def bert_text_preparation(text, tokenizer):
    #Ref. : https://towardsdatascience.com/3-types-of-contextualized-word-embeddings-from-bert-using-transfer-learning-81fcefe3fe6d
    marked_text = "[CLS] " + text + " [SEP]"
    tokenized_text = tokenizer.tokenize(marked_text)
    indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
    segments_ids = [1]*len(indexed_tokens)
    # Convert inputs to PyTorch tensors
    tokens_tensor = torch.tensor([indexed_tokens])
    segments_tensors = torch.tensor([segments_ids])
    return tokenized_text, tokens_tensor, segments_tensors

In [40]:
def get_bert_embeddings(tokens_tensor, segments_tensors, model):
#Ref. : https://towardsdatascience.com/3-types-of-contextualized-word-embeddings-from-bert-using-transfer-learning-81fcefe3fe6d  
    with torch.no_grad():
        outputs = model(tokens_tensor, segments_tensors)
        # Removing the first hidden state
        # The first state is the input state
        hidden_states = outputs[2][1:]
    # Getting embeddings from the final BERT layer
    token_embeddings = hidden_states[-1]
    # Collapsing the tensor into 1-dimension
    token_embeddings = torch.squeeze(token_embeddings, dim=0)
    # Converting torchtensors to lists
    list_token_embeddings = [token_embed.tolist() for token_embed in token_embeddings]
    return list_token_embeddings

In [41]:
texts=[]
temp=[]
for i in range(df['words'].size):
  #print(i)
  if(df['words'][i]!='.'):
    temp.append(df['words'][i])
  if (df['words'][i]=='.'):
    temp.append('.')
    final_sentence = " ".join(temp)
    texts.append(final_sentence)
    temp=[]

In [42]:
pattern = r'#'
tokens=[]
for text in texts :
#if (1==1):
  a=text.split()
  #print(a)
  for i in a:
    #print(i)
    temp=[]
    tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(i, tokenizer)
    temp_tokenized_text = tokenized_text.copy()
    temp_tokenized_text.remove('[CLS]')
    temp_tokenized_text.remove('[SEP]')
    #print(temp_tokenized_text)
    for j in temp_tokenized_text:
      #print(j)
      temp.append(j)
    tokens.append(temp)
#print(tokens)
#len(tokens)
#df['words'].size
  #print(tokens)
for i in range(df['coref'].size):
  df['tokens_per_word'][i]=tokens[i]

In [43]:
large_tokenized_text=[]
large_embeddings=[]
for text in texts:
  #print(text)
  tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
  list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
  #print(tokenized_text) 
  for i in tokenized_text:                                           
    large_tokenized_text.append(i)
  for j in list_token_embeddings:
    large_embeddings.append(j)
    
for i in range(df['tokens_per_word'].size):
  temp_context_vectors=[]
  for j in df['tokens_per_word'][i]:
    #print(j)
    word_index = large_tokenized_text.index(j)
    word_embedding = large_embeddings[word_index]
    temp_context_vectors.append(word_embedding)
  mean_vector=np.mean(temp_context_vectors,axis=0)
  df['embedding_per_token'][i]=mean_vector

In [44]:
#pattern = r'#'
#for text in texts:
  #reading one sentence at a time and generating its tokens
  #as well as token embeddings from above methods
 # tokenized_text, tokens_tensor, segments_tensors = bert_text_preparation(text, tokenizer)
  #list_token_embeddings = get_bert_embeddings(tokens_tensor, segments_tensors, model)
  #print(tokenized_text)

#here we extract the tokens,except the sep and cls and punctuations
#and then we match which token is a substring of which cell
#of the word column in original DF
#when we find a match, we record the index of that item in DF
#then we update the token & embedding column with the tokens & embeddings obtained from that word
#for i in tokenized_text:
 # print(i)
  #if i != "[CLS]" and i != "[SEP]" and i != ".":
   # word_index = tokenized_text.index(i)
    #word_embedding = list_token_embeddings[word_index]
    #print(word_index)
    #if (i!='(' and i!=')'):
      #i = re.sub(pattern, '', i)
      #print(i)
      #x = df.index[df['words'].str.contains(i, case=False)]
      #print(x)
    #print(x)
    #for j in range(len(x)):
      #df['tokens_per_word'][x[j]].append(i)
      #df['tokens_per_word'][x[j]]= i
      #df['embedding_per_token'][x[j]].append(word_embedding)
     # df['embedding_per_token'][x[j]]=word_embedding

In [45]:
#method to read the next occurance of the coref no. for eg. 3
#if it finds that number again, it will add 
#all the words that have occured from start to end to a list
#then pass that list as a return parameter
def add_to_dict(i,number,my_coref_list,my_word_list,df):
  #print('number is',number,'location in df is',i)
  temp_coref = df['coref'][i+1]
  my_coref_list.append(df['embedding_per_token'][i+1])
  my_word_list.append(df['words'][i+1])
  i=i+1
  while number not in temp_coref:
    my_coref_list.append(df['embedding_per_token'][i+1])
    my_word_list.append(df['words'][i+1])
    temp_coref = df['coref'][i+1]
    i = i+1
  return my_coref_list,my_word_list

In [46]:
#This method extracts the number from the alphanumeric mentions
def extract_number(number):
  #print('I am extracting letters from',number)
  alpha_num = number
  res = [i for i in number if i.isdigit()]
  if (len(res)>=2):
    number= ''.join(res)
    #print('final number',number)
  return number

In [47]:
#Main purpose is to have a dictionary and a final dataframe created for all the mentions
#If we have discontinuous mentions, then they will be all added together as one 
#Their context vectors and their tokens will be combined together in a single list

i=0
#coref_groups = {}
word_groups={}
mentions=[]
counter=0
alpha_num=''
#mean_vector=[]
column_names = ["index", "mention_no","context_vectors","tokens"]
inter_df = pd.DataFrame(columns = column_names)
for i in range(df['coref'].size):
  temp = df['coref'][i]
  mentions = temp.split('|')

  #print(mentions)
  for mention in mentions:
    #This is the case 1 where the mention
    #only has a opening and no closing and longer span
    if '(' in mention and ')' not in mention:
      my_coref_list=[]
      my_word_list=[]
      x=mention.split('(')
      number=x[1]
      #if (re.search('[a-zA-Z]', number)):
      #  number=extract_number(number)
      #print('Case 1',number)
      my_coref_list.append(df['embedding_per_token'][i])
      my_word_list.append(df['words'][i])
      #we call the above method which reads all the mentions
      #until it finds the end of the mention previously encountered
      my_coref_list,my_word_list = add_to_dict(i,number,my_coref_list,my_word_list,df)

    elif '(' in mention and ')' in mention:
      #This is the singleton case
      #Where the mention has both opeining and closing
      my_coref_list=[]
      my_word_list=[]
      x = mention.split('(')
      number=x[1].split(')')[0]
      #print('Case 2',number)
      my_coref_list.append(df['embedding_per_token'][i])
      my_word_list.append(df['words'][i])

    else:
      #print('Case 4 not here')
      continue
    inter_df.loc[counter]=[i,number,my_coref_list,my_word_list]
    #coref_groups[number] = my_coref_list
    word_groups[number] = my_word_list
    counter+=1

word_groups
inter_df

,index,mention_no,context_vectors,tokens
0,0,1,"[[-0.22877123951911926, 0.20585937798023224, 0...","[Diabetes, Insipidus, in, Mice, with, a, Mutat..."
1,3,2,"[[-0.10411273688077927, -0.04446271434426308, ...","[Mice, with, a, Mutation, in, Aquaporin, -, 2]"
2,5,3,"[[0.1659092754125595, -0.11474323272705078, -0...","[a, Mutation, in, Aquaporin, -, 2]"
3,8,4,"[[0.03711164277046919, -0.06261884421110153, -...","[Aquaporin, -, 2]"
4,12,5,"[[-0.2006397396326065, -0.16753144562244415, 0...","[Congenital, nephrogenic, diabetes, insipidus]"
...,...,...,...,...
109,616,20,"[[-0.0694167222827673, 0.010410560294985771, -...",[AQP2]
110,629,19,"[[-0.21900470554828644, 0.09637747704982758, -...",[water]
111,636,20,"[[0.0586155503988266, -0.19091956317424774, -0...",[it]
112,640,40,"[[-0.012681677006185055, 0.20679669082164764, ...","[collecting, -, duct, principal, cells]"


In [48]:
#The purpose of this cell is to :-
#1. Merge all the cntext vectors and all the tokens for the discontinuous spans.
#2. Fid the mean of all the context vectors.
#3. Normalize the meaned context vectors obtained in the previous step.
column_names = ["index", "mention_no","mean_normalized_vector","tokens"]
final_df = pd.DataFrame(columns = column_names)
counter=0
for index, row in inter_df.iterrows():
  x=row['mention_no']
  if (re.search('[a-zA-Z]',x)):
    temp_coref_list=[]
    temp_word_list=[]
    current_number=extract_number(x)
    if (current_number not in final_df.values):
      for i in range(inter_df['mention_no'].size):
        temp = inter_df['mention_no'][i]
        if (re.search('[a-zA-Z]',temp)) and (extract_number(temp)) == current_number:
          print('Adding only',inter_df['mention_no'][i],'to the list')
          temp_coref_list+=inter_df['context_vectors'][i]
          temp_word_list+=inter_df['tokens'][i]
      mean_vector=np.mean(temp_coref_list,axis=0)
      norm_mean_vector = mean_vector / np.linalg.norm(mean_vector)
      final_df.loc[counter]=[index,current_number,norm_mean_vector,temp_word_list]
      counter+=1
  else:
    mean_vector=np.mean(row['context_vectors'],axis=0)
    norm_mean_vector = mean_vector / np.linalg.norm(mean_vector)
    final_df.loc[counter]=[row['index'],row['mention_no'],norm_mean_vector,row['tokens']]
    counter+=1


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  


Adding only 3a to the list
Adding only 3a to the list
Adding only 24a to the list
Adding only 24a to the list
Adding only 24b to the list
Adding only 24b to the list
Adding only 28b to the list
Adding only 28b to the list
Adding only 28a to the list
Adding only 28a to the list
Adding only 31a to the list
Adding only 31a to the list


In [49]:
final_df

,index,mention_no,mean_normalized_vector,tokens
0,0,1,"[-0.002128100072283943, 0.00035211816842894246...","[Diabetes, Insipidus, in, Mice, with, a, Mutat..."
1,3,2,"[0.0025281501780507433, -0.0013584542354517226...","[Mice, with, a, Mutation, in, Aquaporin, -, 2]"
2,5,3,"[0.0026430296095377877, -0.0012788860062432866...","[a, Mutation, in, Aquaporin, -, 2]"
3,8,4,"[0.005380736439015781, 0.002337076059069516, -...","[Aquaporin, -, 2]"
4,12,5,"[-0.017618984880443493, 0.0016945110431218614,...","[Congenital, nephrogenic, diabetes, insipidus]"
...,...,...,...,...
99,616,20,"[-0.0047985462672887275, 0.000719647278077975,...",[AQP2]
100,629,19,"[-0.014801009022190803, 0.006513485195577097, ...",[water]
101,636,20,"[0.0039581983385687846, -0.012892440531816853,...",[it]
102,640,40,"[0.005998361372358761, 0.019226518188499176, -...","[collecting, -, duct, principal, cells]"


In [50]:
#Preparing the gold matrix, this is were we group all te mentions together according to the original file.
final_df['temp']=final_df['mention_no']
n=len(final_df.index)
gold_matrix=np.zeros(shape=(n,n))
for i in range(n):
  for j in range (n):
    if (final_df.iloc[[i]]['mention_no'][i] == final_df.iloc[[j]]['temp'][j]):
      gold_matrix[i][j]+=1
gold_matrix

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [51]:
stacked = np.row_stack(final_df['mean_normalized_vector'])
all_pairs_cosine_sim = np.dot(stacked,stacked.T)
print(all_pairs_cosine_sim)
all_pairs_cosine_sim[all_pairs_cosine_sim < 1.0] = 0
all_pairs_cosine_sim[all_pairs_cosine_sim > 1.0] = 1
all_pairs_cosine_sim

[[1.         0.99734218 0.99458091 ... 0.9309727  0.95859348 0.95035466]
 [0.99734218 1.         0.99742103 ... 0.93294872 0.95938933 0.94754529]
 [0.99458091 0.99742103 1.         ... 0.93342533 0.95999023 0.94808994]
 ...
 [0.9309727  0.93294872 0.93342533 ... 1.         0.90850927 0.92079058]
 [0.95859348 0.95938933 0.95999023 ... 0.90850927 1.         0.94874468]
 [0.95035466 0.94754529 0.94808994 ... 0.92079058 0.94874468 1.        ]]


array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
A=all_pairs_cosine_sim.tolist()
A= pd.DataFrame.from_records(A)
A=A.to_numpy().flatten()
B=gold_matrix.tolist()
B = pd.DataFrame.from_records(B)
B=B.to_numpy().flatten()
print('Confusion matrix is :\n',confusion_matrix(B,A))
print('Accuracy is :',accuracy_score(B,A)*100,'%')
print('F1 score is :', f1_score(B,A)*100,'%')

Confusion matrix is :
 [[10298    20]
 [  369   129]]
Accuracy is : 96.40347633136095 %
F1 score is : 39.876352395672335 %


In [53]:
#Gold standard actual graph. Preparing the graph for the gold standard matrix we have obtained in previous step.
#G = nx.DiGraph()
#for i in range (len(final_df.index)):
  #G.add_node(i,label=final_df['tokens'][i])

#for i in range (len(final_df.index)):
 # for j in range(i):
  #  if (final_df['mention_no'][i] == final_df['mention_no'][j]):
   #   G.add_edge(i,j,arrowsize=0.0)

# set defaults
#G.graph['graph']={'rankdir':'TD'}
#G.graph['node']={'shape':'rectangle'}
#G.graph['edges']={'arrowsize':'0.0'}

#A = to_agraph(G)
#print(A)
#A.layout('circo')
#A.draw('gold_standard.png')
#img = mpimg.imread('/content/gold_standard.png')
#imgplot = plt.imshow(img)
#plt.show()

In [54]:
#Predicted graph
#G = nx.DiGraph()
#for i in range (len(all_pairs_cosine_sim)):
  #G.add_node(i,label=final_df['tokens'][i])
  #for j in range (i):
    #if (i!=j) and (all_pairs_cosine_sim[i][j]>0.0):
      #print('hello')
     # G.add_edge(i,j,arrowsize=0.0)

# set defaults
#G.graph['graph']={'rankdir':'TD'}
#G.graph['node']={'shape':'rectangle'}
#G.graph['edges']={'arrowsize':'0.0'}

#A = to_agraph(G)
#print(A)
#A.layout('circo')
#A.draw('predicted_coreference.png')
#img = mpimg.imread('/content/predicted_coreference.png')
#imgplot = plt.imshow(img)
#plt.show()